Данное домашнее задание посвящено схеме базе данных, связанной с футбольным командами. Схема расположена в hw_diagram.jpg
## Описание таблиц 

### Positions 
Позиции игроков
id - уникальный идентификатор позиции 
Name - Название таблицы 

### Coaches 
Таблица тренеров 
id - уникальный идентификатор тренера
Name - имя тренера 

## Teams 
Таблица команд 
id - уникальный идентификатор  команды 
Name - имя команды 
Location - расположение команды 
Coach_id - идентификатор тренера 

### Players 
Таблица игроков 
Id - уникальный идентификатор игрока 
Name - имя игрока 
Age - возраст игрока 
Position_id - идентификатор позиции 
Team_id - идентификатор команды 
Start_date - дата начала игры в команде 

### PlayersArchive 
Архив по игрокам (информация о том, в какой команде игрок играл ранее) 
player_id - идентификатор игрока 
team_id - идентификатор команды 
start_date - дата начала игры 
end_date - дата окончания игры

# Задание 1
Реализовать перенакатываемые sql-скрипты по схеме hw_diagram, приложенного ниже.

In [1]:
%load_ext sql
%sql sqlite:///HomeW.db

'Connected: @HomeW.db'

In [2]:
%%sql

PRAGMA foreign_keys=ON;
DROP TABLE IF EXISTS PlayersArchive;
DROP TABLE IF EXISTS Players;
DROP TABLE IF EXISTS Teams;
DROP TABLE IF EXISTS Coaches;
DROP TABLE IF EXISTS Positions;

CREATE TABLE Positions(
    ID INTEGER PRIMARY KEY,
    Name VARCHAR(50)
);

CREATE TABLE Coaches(
    ID INTEGER PRIMARY KEY,
    Name VARCHAR(50)
);

CREATE TABLE Teams(
    ID INTEGER PRIMARY KEY,
    Name VARCHAR(50),
    Location VARCHAR(50),
    Coach_id INTEGER,
    FOREIGN KEY (Coach_id) REFERENCES Coaches(ID)
);

CREATE TABLE Players(
    ID INTEGER PRIMARY KEY,
    Name VARCHAR(50),
    Position_id INTEGER,
    Age INTEGER,
    Team_id INTEGER,
    Start_date DATE,
    FOREIGN KEY (Position_id) REFERENCES Positions(ID),
    FOREIGN KEY (Team_id) REFERENCES Teams(ID)
);

CREATE TABLE PlayersArchive(
    Player_id INTEGER,
    Team_id INTEGER,
    Start_date DATE,
    End_date DATE,
    PRIMARY KEY (Player_id, Team_id, Start_date),
    FOREIGN KEY (Player_id) REFERENCES Players(ID),
    FOREIGN KEY (Team_id) REFERENCES Teams(ID)
);




 * sqlite:///HomeW.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

# Задание 2
Написать sql-скрипт, осуществляюший добавление всех возможных позиций игроков. (В данной задаче можно ограничится четырьмя - вратарь, защитник, полузащитник, нападающий)

In [3]:
%%sql
INSERT INTO Positions(Name) VALUES ("Vratar"), ("ZaSHITnick"), ("PoluzaSHITnick"), ("Napadaushii")

 * sqlite:///HomeW.db
4 rows affected.


[]

# Задание 3
Написать функцию, осуществляющую добавление тренера. На вход функции подается имя тренера. При невозможности вставки функция должна обработать данное исключение и вывести пользователю информацию о проблеме

После написания функции необходимо добавить несколько тренеров для таблицы.

In [4]:
import sqlite3
DB = sqlite3.connect("HomeW.db")

In [5]:
def addCoach (name):
    CUR = DB.cursor()
    try:
        CUR.execute('''PRAGMA foreign_keys=ON''')
        CUR.execute('''INSERT INTO Coaches(Name) VALUES(?)''', (name,))
        DB.commit()
    except Exception as e:
        print(e)

In [6]:
addCoach("coach1")
addCoach("coach2")

In [7]:
%%sql

SELECT * FROM Coaches;

 * sqlite:///HomeW.db
Done.


ID,Name
1,coach1
2,coach2


# Задание 4

Написать функцию, осуществляющую добавление команды. На вход функция принимает (имя команды, местоположение, имя тренера).
На выходе должна быть добавлена команда.

После написания функции необходимо добавить несколько команд

In [8]:
def addTeam (name, loc, coach_name):
    CUR = DB.cursor()
    try:
        CUR.execute('''PRAGMA foreign_keys=ON''')
        CUR.execute('''SELECT ID FROM Coaches WHERE Name = ?''', (coach_name,))
        CUR.execute('''INSERT INTO Teams(Name, Location, Coach_Id) VALUES(?, ?, ?)''', (name, loc, CUR.fetchone()[0]))
        DB.commit()
    except Exception as e:
        print(e)

In [9]:
addTeam("team1", "loc1", "coach1")
addTeam("team2", "loc2", "coach2")

In [10]:
%%sql
SELECT * FROM Teams;

 * sqlite:///HomeW.db
Done.


ID,Name,Location,Coach_id
1,team1,loc1,1
2,team2,loc2,2


# Задание 5

Написать функцию, осуществляющую обновление информации о команде. У функции на входе должен быть один обязательный параметр - название команды и два необязательных параметра - имя тренера и местоположение. При этом хотя бы один из необязательных параметров должен быть заполнен. Функция должна обновлять значение переданных необязательных параметров.

После написания функции вызовите данную функцию несколько раз

In [11]:
def updateTeam (name, coach_name = None, loc = None):
    CUR = DB.cursor() 
    try:
        CUR.execute('''PRAGMA foreign_keys=ON''')
        query = ""
        if coach_name != None:
            CUR.execute('''SELECT ID FROM Coaches WHERE Name = ?''', (coach_name,))
            query += "Coach_id = " + str(CUR.fetchone()[0])
            if loc != None:
                query += ","
            query += "\n"
        if loc != None:
            query += "Location = " + str(loc) + "\n"
        query = "UPDATE Teams SET " + query + "WHERE Name = '" + str(name) + "'"
        CUR.execute(query)
        DB.commit()
    except Exception as e:
        print(e)

In [12]:
updateTeam("team1", "Foo")

In [12]:
%%sql
SELECT * FROM Teams;

 * sqlite:///HomeW.db
Done.


ID,Name,Location,Coach_id
1,team1,loc1,1
2,team2,loc2,2


# Задание 6

Написать функцию, осуществляющую добавление нового футболиста в таблицу игроков. На входе (имя, возраст, название позиции, название команды, дата перехода в команду(необязательный параметр)). Если дата перехода не указана, то нужно использовать текущую дату. Предусмотреть различные ошибки (такие как неверное имя позиции, неверное название команды, возраст < 10 и > 50 и т.п.)

После написания функции необходимо добавить несколько игроков.

In [13]:
import datetime

def checkPos (pos):
    CUR = DB.cursor()
    try:
        CUR.execute('''SELECT ID FROM Positions WHERE Name = ?''', (pos,))
        res = CUR.fetchone()[0]
        if res:
            return res
    except Exception as e:
        print(e)
    return None
            
def checkTeam (name):
    CUR = DB.cursor()
    try:
        CUR.execute('''SELECT ID FROM Teams WHERE Name = ?''', (name,))
        res = CUR.fetchone()
        if res:
            return res[0]
    except Exception as e:
        print(e)
    return None

def checkAge (age):
    if age >= 10 and age <= 50:
        return True
    return False

def processDate (date):
    if date == None:
        return datetime.datetime.now()
    try:
        res = datetime.datetime.strptime(date, "%Y-%m-%d %H:%M:%S")
    except:
        print(res)
        res = None
    return res

def checkPlayer (name):
    CUR = DB.cursor()
    try:
        CUR.execute('''SELECT ID FROM Players WHERE Name = ?''', (name,))
        res = CUR.fetchone()
        if res:
            return res[0]
    except Exception as e:
        print(e)
    return None

def addPlayer (name, age, pos, team_name, start_date = None):
    CUR = DB.cursor()
    CUR.execute('''PRAGMA foreign_keys=ON''')
    pos_id = checkPos(pos)
    if not pos_id:
        print("Bad position!")
        return
    team_id = checkTeam(team_name)
    if not team_id:
        print("Bad team!")
        return
    if not checkAge(age):
        print("Bad age!")
        return
    date = processDate(start_date)
    if not date:
        print("Bad date format!")
        return
    player_id = checkPlayer(name)
    if not player_id:
        CUR.execute('''INSERT INTO Players(Name, Position_id, Age, Team_id, Start_date) VALUES(?, ?, ?, ?, ?)''',
                    (name, pos_id, age, team_id, date))
    else:
        old_team_id = CUR.execute('''SELECT Team_id FROM Players WHERE id = ?''', (player_id,)).fetchone()[0]
        old_date = CUR.execute('''SELECT Start_date FROM Players WHERE id = ?''', (player_id,)).fetchone()[0]
        CUR.execute("INSERT INTO PlayersArchive(Player_id, Team_id, Start_date, End_date) VALUES(?, ?, ?, ?)",
                    (player_id, old_team_id, old_date, date))
        CUR.execute("UPDATE Players SET Position_id = ?, Age = ?, Team_id = ?, Start_date = ? WHERE ID = ?",
                    (pos_id, age, team_id, date, player_id))
    DB.commit()
    


In [14]:
%%sql
SELECT * FROM Players;

 * sqlite:///HomeW.db
Done.


ID,Name,Position_id,Age,Team_id,Start_date


In [15]:
%%sql
SELECT * FROM PlayersArchive;

 * sqlite:///HomeW.db
Done.


Player_id,Team_id,Start_date,End_date


In [16]:
addPlayer("player1", 20, "Vratar", "team1")
addPlayer("player2", 20, "Vratar", "team2")

# Задание 7
Написать функцию для перевода игрока в другую команду. Данная функция должна осуществить архивирование текущей записи в таблице игроков в таблицу players_archive с заполнение end_date - датой начала игры за новую команду - 1 день. А также осуществлять изменение поля в таблице Players. При возникновении ошибки должен осуществляться полный откат транзакции.

In [17]:
%%sql
SELECT * FROM Players;

 * sqlite:///HomeW.db
Done.


ID,Name,Position_id,Age,Team_id,Start_date
1,player1,1,20,1,2019-03-30 14:58:25.626998
2,player2,1,20,2,2019-03-30 14:58:25.641547


In [18]:
%%sql
SELECT * FROM PlayersArchive;

 * sqlite:///HomeW.db
Done.


Player_id,Team_id,Start_date,End_date


In [19]:
addPlayer("player1", 20, "Vratar", "team2")
addPlayer("player2", 20, "Vratar", "team1")

In [20]:
%%sql
SELECT * FROM Players;

 * sqlite:///HomeW.db
Done.


ID,Name,Position_id,Age,Team_id,Start_date
1,player1,1,20,2,2019-03-30 14:59:17.421336
2,player2,1,20,1,2019-03-30 14:59:17.431717


In [21]:
%%sql
SELECT * FROM PlayersArchive;

 * sqlite:///HomeW.db
Done.


Player_id,Team_id,Start_date,End_date
1,1,2019-03-30 14:58:25.626998,2019-03-30 14:59:17.421336
2,2,2019-03-30 14:58:25.641547,2019-03-30 14:59:17.431717


In [22]:
addPlayer("player1", 20, "Vratar", "team1")
addPlayer("player2", 20, "Vratar", "team2")

In [23]:
%%sql
SELECT * FROM Players;

 * sqlite:///HomeW.db
Done.


ID,Name,Position_id,Age,Team_id,Start_date
1,player1,1,20,1,2019-03-30 15:00:07.604076
2,player2,1,20,2,2019-03-30 15:00:07.665447


In [24]:
%%sql
SELECT * FROM PlayersArchive;

 * sqlite:///HomeW.db
Done.


Player_id,Team_id,Start_date,End_date
1,1,2019-03-30 14:58:25.626998,2019-03-30 14:59:17.421336
2,2,2019-03-30 14:58:25.641547,2019-03-30 14:59:17.431717
1,2,2019-03-30 14:59:17.421336,2019-03-30 15:00:07.604076
2,1,2019-03-30 14:59:17.431717,2019-03-30 15:00:07.665447


In [25]:
addPlayer("player1", 20, "Vratar", "team2")

In [26]:
%%sql
SELECT * FROM PlayersArchive;

 * sqlite:///HomeW.db
Done.


Player_id,Team_id,Start_date,End_date
1,1,2019-03-30 14:58:25.626998,2019-03-30 14:59:17.421336
2,2,2019-03-30 14:58:25.641547,2019-03-30 14:59:17.431717
1,2,2019-03-30 14:59:17.421336,2019-03-30 15:00:07.604076
2,1,2019-03-30 14:59:17.431717,2019-03-30 15:00:07.665447
1,1,2019-03-30 15:00:07.604076,2019-03-30 15:01:34.562255


In [27]:
addPlayer("player1", 20, "Vratar", "team1", "2019-03-30 14:30:00")

In [28]:
%%sql
SELECT * FROM Players;

 * sqlite:///HomeW.db
Done.


ID,Name,Position_id,Age,Team_id,Start_date
1,player1,1,20,1,2019-03-30 14:30:00
2,player2,1,20,2,2019-03-30 15:00:07.665447
